In [8]:
# Step 1: Import necessary modules
import os
import arcpy

# Step 2: User set geodatabase location
gdb_path = input("Enter the full path to the geodatabase, must end in .gdb: ")

# Step 3: Validate the geodatabase path
if not os.path.exists(gdb_path) or not gdb_path.endswith(".gdb"):
    raise ValueError("The path provided is not a valid file geodatabase")

# Step 4: If the path is valid, set the workspace
arcpy.env.workspace = gdb_path
print(f"Workspace set to: {gdb_path}")

# Step 5: Prompt the user for theme and place keywords
theme_keywords = input("Enter theme keywords, separated by commas: ")
place_keywords = input("Enter place keywords, separated by commas: ")

# Step 6: Prompt the user for the location to save the XML metadata file
xml_path = input("Enter the full path to the folder where the XML metadata file should be saved: ")

# Step 6: Create a new XML metadata file
try:
    with open(os.path.join(xml_path, 'metadata.xml'), 'w') as xml_file:
        xml_content = """<?xml version='1.0' encoding='UTF-8'?>
<metadata>
  <idinfo>
    <keywords>
      <theme>
        <themekt>None</themekt>
        <themekey>{theme_keywords}</themekey>
      </theme>
      <place>
        <placekt>None</placekt>
        <placekey>{place_keywords}</placekey>
      </place>
    </keywords>
  </idinfo>
</metadata>
""".format(
            theme_keywords=theme_keywords,
            place_keywords=place_keywords
        )
        xml_file.write(xml_content)
    print(f"XML metadata file created at: {xml_path}")
except Exception as e:
    raise ValueError(f"Failed to create the XML file: {e}")

# Step 7: Function to update metadata for each item in the geodatabase
def update_metadata(item_path, theme_keywords, place_keywords):
    # Import metadata from the XML template
    arcpy.metadata.Metadata(item_path).importMetadata(xml_path)

    # Update theme and place keywords
    md = arcpy.metadata.Metadata(item_path)
    md.themeKeywords = theme_keywords
    md.placeKeywords = place_keywords
    md.save()
    print(f"Metadata updated for: {item_path}")

# Step 8: Update metadata for the geodatabase itself
update_metadata(gdb_path, theme_keywords, place_keywords)

# Step 9: Iterate through feature datasets and update metadata
for dataset in arcpy.ListDatasets():
    dataset_path = os.path.join(gdb_path, dataset)
    update_metadata(dataset_path, theme_keywords, place_keywords)

# Step 10: Iterate through feature classes and update metadata
for feature_class in arcpy.ListFeatureClasses():
    feature_class_path = os.path.join(gdb_path, feature_class)
    update_metadata(feature_class_path, theme_keywords, place_keywords)

# Step 11: Iterate through tables and update metadata
for table in arcpy.ListTables():
    table_path = os.path.join(gdb_path, table)
    update_metadata(table_path, theme_keywords, place_keywords)

print("Metadata update process completed.")


Enter the full path to the geodatabase, must end in .gdb: C:\Working\GeMS\Metadata\Test\Test.gdb
Workspace set to: C:\Working\GeMS\Metadata\Test\Test.gdb
Enter theme keywords, separated by commas: theme3,theme4
Enter place keywords, separated by commas: place3,place4
Enter the full path to the folder where the XML metadata file should be saved: C:\Working\GeMS\Metadata\Test
XML metadata file created at: C:\Working\GeMS\Metadata\Test
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb\GeologicMap
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb\CorrelationOfMapUnits
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb\DescriptionOfMapUnits
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb\DataSources
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb\Glossary
Metadata updated for: C:\Working\GeMS\Metadata\Test\Test.gdb\GeoMaterialDict
Metadata update process completed.
